In [1]:
import numpy as np
import pandas as pd

In [2]:
def obtener_semana(week):
    week = str(week)
    # Traemos los dataframe
    all_info = pd.read_csv('./data/raw/w'+week+'/metrocuadrado_all_w'+week+'.csv')
    all_info.drop_duplicates(inplace = True)
    furnished = pd.read_csv('./data/raw/w'+week+'/metrocuadrado_furnished_w'+week+'.csv')
    furnished.drop_duplicates(inplace = True)
    
    # Merge que identifica furnished dentro de all_info
    df = pd.merge(all_info,furnished, on = ['property_type|rent_type|location','details'
                                            ,'url','surface','price'],
                  how= 'outer', indicator=True)
    df.rename(columns={'n_rooms_x': 'n_rooms', 'n_bath_x': 'n_bath',
                       'metrocuadrado_index_x': 'metro_cuadrado'},inplace=True)
    
    # Indices que solo hay en furnished
    indices = df[df._merge == 'right_only'].index
    for i in indices:
        df.loc[i,'n_rooms'] = df.loc[i,'n_rooms_y']
        df.loc[i,'n_bath'] = df.loc[i,'n_bath_y']
        df.loc[i,'metro_cuadrado'] = df.loc[i,'metrocuadrado_index_y']
    
    # Dropeamos los _y
    df.drop(['n_rooms_y','n_bath_y','metrocuadrado_index_y'],axis=1,inplace=True)
    
    # Dropeamos los duplicados y reseatmos indice
    df.drop_duplicates(inplace = True)
    df.reset_index(drop=True,inplace=True)
    
    return df

In [3]:
def append_dataframes():
    week_list = range(13,18)
    
    df = pd.DataFrame()
    
    for week in week_list:
        df = df.append(obtener_semana(week))
    df.drop_duplicates(inplace = True)
    return df

In [149]:
# Iniciar Data frame
df=append_dataframes()
# limpiar columna de precio y ahora en formato float
df['price']=df.price.replace('[\$,.]','',regex=True).astype(float)
#df['n_rooms']=pd.to_numeric(df['n_rooms'])
sum(df['n_rooms']=='5+')

9

In [118]:
# Crear data frame y split de 

df['price']=df['price'].replace('[\$,]','',regex=True).replace('.','',regex=True).astype(float)

ValueError: could not convert string to float: 

Si hay furnished que no hay en all, identificados como 'right_only' de la columna _merge

In [110]:
# split de columna
s=pd.Series(df.keys())
split=s.str.split('|',expand=True)
df[['property_type+rent_type','location']]=df['property_type|rent_type|location'].str.split(',',expand=True)
df[['property_type','rent_type']]=df['property_type+rent_type'].str.split(' en ',expand=True)
df=df.drop(['property_type|rent_type|location','property_type+rent_type'],axis=1)


In [111]:
df

,price,n_rooms,n_bath,surface,details,url,metro_cuadrado,_merge,location,property_type,rent_type
0,$1.050.000,3,2,63.0 m2,Excelente casa cerca de la calle 80 y Avenida ...,https://www.metrocuadrado.com/inmueble/arriend...,25.6,left_only,LA SOLEDAD NORTE Bogotá D.C..,Casa,Arriendo
1,$3.930.000,3,2,100.0 m2,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,both,CIUDAD SALITRE SUR-ORIENTAL Bogotá D.C..,Casa,Arriendo
2,$750.000,3,1,90.0 m2,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2,left_only,VILLA DE LOS ALPES II SECTOR Bogotá D.C..,Casa,Arriendo
3,$3.400.000,5,4,71.0 m2,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0,left_only,PARIS GAITAN Bogotá D.C..,Casa,Arriendo
4,$1.560.000,4,3,96.0 m2,"Excelente casa de tres niveles, buena ubicació...",https://www.metrocuadrado.com/inmueble/arriend...,47.1,left_only,BALCONES DE ORIENTE Bogotá D.C..,Casa,Arriendo
5,$1.100.000,3,2,69.0 m2,"Casa en conjunto cerrado, recién remodelada, c...",https://www.metrocuadrado.com/inmueble/arriend...,32.4,left_only,TIBABUYES Bogotá D.C..,Casa,Arriendo
6,$3.400.000,4,4,71.0 m2,casa ezquinera tres niveles excelente ubicacio...,https://www.metrocuadrado.com/inmueble/arriend...,60.9,left_only,PARIS GAITAN Bogotá D.C..,Casa,Arriendo
7,$1.400.000,4,2,82.0 m2,casa en conjunto cerrado consta de tres pisos ...,https://www.metrocuadrado.com/inmueble/arriend...,52.9,left_only,PORTALES DEL NORTE Bogotá D.C..,Casa,Arriendo
8,$1.200.000,3,2,72.0 m2,Cómoda casa en sector residencial de Ciudadela...,https://www.metrocuadrado.com/inmueble/arriend...,40.9,left_only,URBANIZACION BOLIVIA ORIENTAL Bogotá D.C..,Casa,Arriendo
9,$1.400.000,4,2,100.0 m2,Casa en arriendo de 2 niveles en La Isabela 2 ...,https://www.metrocuadrado.com/inmueble/arriend...,33.3,left_only,URBANIZACION AZAFRANES Bogotá D.C..,Casa,Arriendo
